In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-03-07 09:19:59--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.6’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.27MB/s    in 0.2s    

2022-03-07 09:19:59 (5.27 MB/s) - ‘postgresql-42.2.9.jar.6’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DateType

In [9]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Gift_Card_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
schema = StructType([
  StructField("marketplace",StringType(),True),
  StructField("customer_id",IntegerType(),True),
  StructField("review_id",StringType(),True),
  StructField("product_id", StringType(), True),
  StructField("product_parent",IntegerType(),True),
  StructField("product_title",StringType(),True),
  StructField("product_category",StringType(),True),
  StructField("star_rating",IntegerType(),True),
  StructField("helpful_votes",IntegerType(),True),
  StructField("total_votes",IntegerType(),True),
  StructField("vine",StringType(),True),
  StructField("verified_purchase",StringType(),True),
  StructField("review_headline",StringType(),True),
  StructField("review_body",StringType(),True),
  StructField("review_date",DateType(),True)
])
user_data_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Gift_Card_v1_00.tsv.gz"), sep="\t", header=True, schema=schema, timestampFormat="yyyy-MM-dd")

# Show DataFrame
user_data_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   24371595|R27ZP1F1CD0C3Y|B004LLIL5A|     346014806|Amazon eGift Card...|       Gift Card|          5|            0|          0|   N|                Y|          Five Stars|Great birthday gi...| 2015-08-31|
|         US|   42489718| RJ7RSBCHUDNNE|B004LLIKVU|     473048287|Amazon.com eGift ...|       Gift Card|          5|    

In [11]:
user_data_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



In [12]:
user_data_df = user_data_df.dropna()

In [13]:
reviews_df = user_data_df.select("review_id", "customer_id", "product_id", "product_parent", "review_date")
reviews_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R27ZP1F1CD0C3Y|   24371595|B004LLIL5A|     346014806| 2015-08-31|
| RJ7RSBCHUDNNE|   42489718|B004LLIKVU|     473048287| 2015-08-31|
|R1HVYBSKLQJI5S|     861463|B00IX1I3G6|     926539283| 2015-08-31|
|R2HAXF0IIYQBIR|   25283295|B00IX1I3G6|     926539283| 2015-08-31|
| RNYLPX611NB7Q|     397970|B005ESMGV4|     379368939| 2015-08-31|
|R3ALA9XXMBEDZR|   18513645|B004KNWWU4|     326384774| 2015-08-31|
|R3R8PHAVJFTPDF|   22484620|B004LLIKVU|     473048287| 2015-08-31|
|R18WWEK8OIXE30|   14765851|BT00CTP2EE|     775486538| 2015-08-31|
|R1EGUNQON2J277|   18751931|B004LLIKVU|     473048287| 2015-08-31|
|R21Z4M4L98CPU2|   15100528|B004W8D102|     595099956| 2015-08-31|
| R6JH7A117FHFA|    3559726|B004LLIKVU|     473048287| 2015-08-31|
|R1XZHS8M1GCGI7|   23413911|B004KNWWU4|     326384774| 2015-08

In [14]:
reviews_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [15]:
products_df = user_data_df.select("product_id", "product_title")
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B004LLIL5A|Amazon eGift Card...|
|B004LLIKVU|Amazon.com eGift ...|
|B00IX1I3G6|Amazon.com Gift C...|
|B00IX1I3G6|Amazon.com Gift C...|
|B005ESMGV4|Amazon.com Gift C...|
|B004KNWWU4|Amazon Gift Card ...|
|B004LLIKVU|Amazon.com eGift ...|
|BT00CTP2EE|Amazon.com Gift C...|
|B004LLIKVU|Amazon.com eGift ...|
|B004W8D102|Amazon Gift Card ...|
|B004LLIKVU|Amazon.com eGift ...|
|B004KNWWU4|Amazon Gift Card ...|
|B004LLIKVU|Amazon.com eGift ...|
|B004LLIKVU|Amazon.com eGift ...|
|B00H5BNLUS|Amazon eGift Card...|
|B004KNWX6C|Amazon Gift Card ...|
|BT00CTOYC0|Amazon.com $15 Gi...|
|B004LLIKVU|Amazon.com eGift ...|
|B00H5BMH44|Amazon eGift Card...|
|B005ESMMKE|Amazon.com Gift C...|
+----------+--------------------+
only showing top 20 rows



In [16]:
products_df=products_df.distinct()

In [17]:
customers_df = user_data_df.groupBy('customer_id').count()
customers_df = customers_df.withColumnRenamed("count", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   12819130|             2|
|   50079826|             2|
|    3399669|             1|
|   23344898|             1|
|   42139031|             1|
|   48994143|             1|
|     768171|             1|
|   29907188|             1|
|   28161223|             1|
|     184483|             1|
|     225566|             1|
|   42628722|             1|
|    2903234|             1|
|   11285702|             2|
|   15423461|             1|
|   30717827|             1|
|   35594973|             1|
|   20157057|             1|
|   27747830|             1|
|   11150152|             1|
+-----------+--------------+
only showing top 20 rows



In [18]:
mode = "append"
jdbc_url="jdbc:postgresql://{endpoint}:5432/{database}"
config = {"user":"root", 
          "password": "password", 
          "driver":"org.postgresql.Driver"}

In [19]:
reviews_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [20]:
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [22]:
customers_df.write.jdbc(url=jdbc_url, table='customers1', mode=mode, properties=config)